# Get Data

In [1]:
import pandas as pd
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split


cwd = os.path.abspath('../news Classifier/CSV Data for Classifier') 
files = os.listdir(cwd) 
df = pd.DataFrame()
for file in files:
    if file.endswith('.csv'):
        df = df.append(pd.read_csv(cwd+"/"+file), ignore_index=True) 
        
df = df.dropna()
df.columns
X = df.drop(['Unnamed: 0', 'date', 'location', 'news title', 'news source(url)',
       'keywords', 'class_name', 'new_class_name'], axis=1)

y = df['new_class_name']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
X_train

,content summary
5780,SINGAPORE - The Asia-Pacific unit of Swiss ban...
9837,KUALA LUMPUR (THE STAR/ASIA NEWS NETWORK) - Th...
12338,Elton John to end Vegas residencyBritish singe...
12220,SINGAPORE - The Land Transport Authority (LTA)...
10214,United States President Donald Trump and visit...
...,...
1063,NEW YORK (BLOOMBERG) -Crude oil climbed along ...
5908,RIO DE JANEIRO (NYTIMES) - Former vice-preside...
13082,BAGHDAD (AFP) - Rocket and drone attacks on We...
12971,THE HAGUE (AFP) - Two former Serbian spy chief...


In [2]:
# import the BaseEstimator
import re 
from sklearn.base import BaseEstimator
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# define the class FeatureEngineering
# This will be our custom transformer that will create 3 new binary columns
# custom transformer must have methods fit and transform
class FeatureEngineering(BaseEstimator):

    def __init__(self):
        pass

    def fit(self, documents, y=None):
        return self
    
        
    def transform(self, x_dataset):
        """
        Function: split text into words and return the root form of the words
        Args:
          text(str): the article
        Return:
          lem(list of str): a list of the root form of the article words
        """
        def preprocess(text):

            # Normalize text
            text = re.sub(r"[^a-zA-Z]", " ", str(text).lower())

            # Tokenize text
            token = word_tokenize(text)

            # Remove stop words
            stop = stopwords.words("english")
            new_stop_words_list = ['said', 'us', 'also', 'mr']
            stop.extend(new_stop_words_list)
            words = [t for t in token if t not in stop]

            # Lemmatization
            lem = [WordNetLemmatizer().lemmatize(w) for w in words]

            return lem
        
        x_dataset.head()

        x_dataset["Preprocessed_Text"] = x_dataset['content summary'].apply(lambda x: preprocess(x))
        x_dataset['Preprocessed_Text2'] = x_dataset['Preprocessed_Text'].apply(' '.join)
        
        
        return x_dataset

In [3]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from mlxtend.feature_selection import ColumnSelector


preprocessor = Pipeline(steps=[('feature engineering', FeatureEngineering()), 
                               ('col_selector', ColumnSelector(cols=('Preprocessed_Text2'),drop_axis=True)),
                               ('tfidf',TfidfVectorizer()),
                            ])


train_features = preprocessor.fit(X_train)
test_features = preprocessor.fit(X_test)

train_features = tf_vec.fit(X_train)
train_features = tf_vec.transform(X_train)
test_features = tf_vec.transform(X_test)

<ipython-input-2-467e96f9d59d>:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_dataset["Preprocessed_Text"] = x_dataset['content summary'].apply(lambda x: preprocess(x))
<ipython-input-2-467e96f9d59d>:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_dataset['Preprocessed_Text2'] = x_dataset['Preprocessed_Text'].apply(' '.join)


In [8]:
train_features

Pipeline(steps=[('feature engineering', FeatureEngineering()),
                ('col_selector',
                 ColumnSelector(cols='Preprocessed_Text2', drop_axis=True)),
                ('tfidf', TfidfVectorizer())])

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost 
from sklearn.metrics  import classification_report
from sklearn import metrics
import time

def fit_eval_model(model, train_features, y_train, test_features, y_test):
    
    """
    Function: train and evaluate a machine learning classifier.
    Args:
      model: machine learning classifier
      train_features: train data extracted features
      y_train: train data lables
      test_features: train data extracted features
      y_test: train data lables
    Return:
      results(dictionary): a dictionary of the model training time and classification report
    """
    results ={}
    
    # Start time
    start = time.time()
    # Train the model
    model.fit(train_features, y_train)
    # End time
    end = time.time()
    # Calculate the training time
    results['train_time'] = end - start
    
    # Test the model
    train_predicted = model.predict(train_features)
    test_predicted = model.predict(test_features)
    
     # Classification report
    results['classification_report'] = classification_report(y_test, test_predicted)
        
    return results

In [7]:
sv = svm.SVC()
ab = AdaBoostClassifier(random_state = 1)
gb = GradientBoostingClassifier(random_state = 1)
xgb = xgboost.XGBClassifier(random_state = 1)
tree = DecisionTreeClassifier()
nb = MultinomialNB()


# Fit and evaluate models
results = {}
for cls in [sv, ab, gb, xgb, tree, nb]:
    cls_name = cls.__class__.__name__
    results[cls_name] = {}
    results[cls_name] = fit_eval_model(cls, train_features, y_train, test_features, y_test)

TypeError: float() argument must be a string or a number, not 'FeatureEngineering'